In [1]:
import json
import os
import pandas as pd
import numpy as np

In [2]:
path = "../data/raw/2023-04-14-job-search/2023-04-14-job-search-location-DC"
file_names = os.listdir(path)

df_list = []

for name in file_names:
    file_path = os.path.join(path, name)
    json_file = json.load(open(file_path))
    if 'jobs_results' in json_file:
        df_json = pd.DataFrame(json_file['jobs_results'])
        df_json['search'] = name.split(".")[0]
        df_json['search_location'] = 'DC'
        df_list.append(df_json)

df_dc = pd.concat(df_list, ignore_index=True)

In [3]:
df_highlights = df_dc['job_highlights'].apply(pd.Series)
df_highlights.columns = ['Qualification', 'Responsiblilites', 'Benefits']

df_highlights = df_highlights.applymap(lambda x: x.get('items') if isinstance(x, dict) else ["None"])
df_highlights = df_highlights.applymap(lambda x: ''.join(x))

df_dc_1 = pd.concat([df_dc, df_highlights], axis=1)

df_dc_1['wfh'] = df_dc['detected_extensions'].apply(lambda x: x.get('work_from_home') == True)
df_dc_1['schedule_type'] = df_dc['detected_extensions'].apply(lambda x: x.get('schedule_type'))


In [4]:
path = "../data/raw/2023-04-14-job-search/2023-04-14-job-search-location-USA"
file_names = os.listdir(path)

df_list = []

for name in file_names:
    file_path = os.path.join(path, name)
    json_file = json.load(open(file_path))
    if 'jobs_results' in json_file:
        df_json = pd.DataFrame(json_file['jobs_results'])
        df_json['search'] = name.split(".")[0]
        df_json['search_location'] = 'USA'
        df_list.append(df_json)

df_usa = pd.concat(df_list, ignore_index=True)

In [5]:
df_highlights = df_usa['job_highlights'].apply(pd.Series)
df_highlights.columns = ['Qualification', 'Responsiblilites', 'Benefits']

df_highlights = df_highlights.applymap(lambda x: x.get('items') if isinstance(x, dict) else ["None"])
df_highlights = df_highlights.applymap(lambda x: ''.join(x))

df_usa_1 = pd.concat([df_usa, df_highlights], axis=1)

df_usa_1['wfh'] = df_usa_1['detected_extensions'].apply(lambda x: x.get('work_from_home') == True)
df_usa_1['schedule_type'] = df_usa_1['detected_extensions'].apply(lambda x: x.get('schedule_type'))


In [6]:
df_all = pd.concat([df_dc_1, df_usa_1])

In [7]:
df_dc_1.to_csv("../data/cleaned/dc_job.csv")
df_usa_1.to_csv("../data/cleaned/usa_job.csv")
df_all.to_csv('../data/cleaned/all_job.csv')